<a href="https://colab.research.google.com/github/zetaqubit/udlbook/blob/main/Notebooks/Chap12/12_1_Self_Attention_soln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.1: Self Attention**

This notebook builds a self-attention mechanism from scratch, as discussed in section 12.2 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [1]:
import numpy as np
import matplotlib.pyplot as plt

The self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [2]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 3
# Number of dimensions of each input
D = 4
# Create an empty list
all_x = []
# Create elements x_n and append to list
for n in range(N):
  all_x.append(np.random.normal(size=(D,1)))
# Print out the list
print(all_x)


[array([[ 1.78862847],
       [ 0.43650985],
       [ 0.09649747],
       [-1.8634927 ]]), array([[-0.2773882 ],
       [-0.35475898],
       [-0.08274148],
       [-0.62700068]]), array([[-0.04381817],
       [-0.47721803],
       [-1.31386475],
       [ 0.88462238]])]


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [3]:
# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters
omega_q = np.random.normal(size=(D,D))
omega_k = np.random.normal(size=(D,D))
omega_v = np.random.normal(size=(D,D))
beta_q = np.random.normal(size=(D,1))
beta_k = np.random.normal(size=(D,1))
beta_v = np.random.normal(size=(D,1))

Now let's compute the queries, keys, and values for each input

In [7]:
# Make three lists to store queries, keys, and values
all_queries = []
all_keys = []
all_values = []
# For every input
for x in all_x:
  # TODO -- compute the keys, queries and values.
  # Replace these three lines
  query = omega_q @ x + beta_q
  key = omega_k @ x + beta_k
  value = omega_v @ x + beta_v

  all_queries.append(query)
  all_keys.append(key)
  all_values.append(value)

We'll need a softmax function (equation 12.5) -- here, it will take a list of arbitrary numbers and return a list where the elements are non-negative and sum to one


In [26]:
def softmax(items_in):

  # TODO Compute the elements of items_out
  # Replace this line
  items_out = np.exp(items_in)
  items_out /= items_out.sum()

  return items_out ;

Now compute the self attention values:

In [41]:
# Create emptymlist for output
all_x_prime = []

# For each output
for n in range(N):
  # Create list for dot products of query N with all keys
  all_km_qn = []
  # Compute the dot products
  for key in all_keys:
    # TODO -- compute the appropriate dot product
    # Replace this line
    dot_product = np.sum([k * q for k, q in zip(key, all_queries[n])])

    # Store dot product
    all_km_qn.append(dot_product)

  # Compute dot product
  attention = softmax(all_km_qn)
  # Print result (should be positive sum to one)
  print("Attentions for output ", n)
  print(attention)

  # TODO: Compute a weighted sum of all of the values according to the attention
  # (equation 12.3)
  # Replace this line
  x_prime = np.zeros((D, 1))
  for w, v in zip(attention, all_values):
    x_prime += w * v

  all_x_prime.append(x_prime)


# Print out true values to check you have it correct
print("x_prime_0_calculated:", all_x_prime[0].transpose())
print("x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]")
print("x_prime_1_calculated:", all_x_prime[1].transpose())
print("x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]")
print("x_prime_2_calculated:", all_x_prime[2].transpose())
print("x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]")


Attentions for output  0
[1.24326146e-13 9.98281489e-01 1.71851130e-03]
Attentions for output  1
[2.79525306e-12 5.85506360e-03 9.94144936e-01]
Attentions for output  2
[0.00505708 0.00654776 0.98839516]
x_prime_0_calculated: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_1_calculated: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_2_calculated: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]
x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]


Now let's compute the same thing, but using matrix calculations.  We'll store the $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ in the columns of a $D\times N$ matrix, using equations 12.6 and 12.7/8.

Note:  The book uses column vectors (for compatibility with the rest of the text), but in the wider literature it is more normal to store the inputs in the rows of a matrix;  in this case, the computation is the same, but all the matrices are transposed and the operations proceed in the reverse order.

In [15]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax

In [22]:
 # Now let's compute self attention in matrix form
def self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):

  # TODO -- Write this function
  # 1. Compute queries, keys, and values
  # 2. Compute dot products
  # 3. Apply softmax to calculate attentions
  # 4. Weight values by attentions
  # Replace this line
  q = omega_v @ X + beta_q
  k = omega_k @ X + beta_k
  v = omega_v @ X + beta_v
  dots = k.T @ q
  attn = softmax_cols(dots)
  print(f'Attention matrix:\n{attn}\n')
  X_prime = v @ attn

  return X_prime

In [23]:
# Copy data into matrix
X = np.zeros((D, N))
X[:,0] = np.squeeze(all_x[0])
X[:,1] = np.squeeze(all_x[1])
X[:,2] = np.squeeze(all_x[2])

# Run the self attention mechanism
X_prime = self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

Attention matrix:
[[6.00720964e-29 1.68351606e-03 1.00000000e+00]
 [9.99002402e-01 5.40805755e-02 7.70546885e-13]
 [9.97597814e-04 9.44235908e-01 9.45457590e-11]]

[[ 0.94693789  1.6009249  -2.71096662]
 [-0.24359964 -0.086292    3.5538184 ]
 [-0.91669344  3.76905648 -6.92955207]
 [-0.44714185  2.05018524 -3.0352825 ]]


If you did this correctly, the values should be the same as above.

TODO:  

Print out the attention matrix
You will see that the values are quite extreme (one is very close to one and the others are very close to zero.  Now we'll fix this problem by using scaled dot-product attention.

In [24]:
# Now let's compute self attention in matrix form
def scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):

  # TODO -- Write this function
  # 1. Compute queries, keys, and values
  # 2. Compute dot products
  # 3. Scale the dot products as in equation 12.9
  # 4. Apply softmax to calculate attentions
  # 5. Weight values by attentions
  # Replace this line
  q = omega_v @ X + beta_q
  k = omega_k @ X + beta_k
  v = omega_v @ X + beta_v
  scaled_dots = k.T @ q / D**0.5
  attn = softmax_cols(scaled_dots)
  print(f'Attention matrix:\n{attn}\n')
  X_prime = v @ attn

  return X_prime

In [25]:
# Run the self attention mechanism
X_prime = scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

Attention matrix:
[[7.51694838e-15 3.29483974e-02 9.99989399e-01]
 [9.69367466e-01 1.86743853e-01 8.77798694e-07]
 [3.06325336e-02 7.80307749e-01 9.72336127e-06]]

[[ 9.67678489e-01  1.37185441e+00 -2.71092104e+00]
 [-2.38858164e-01  6.21096251e-03  3.55377970e+00]
 [-7.69157406e-01  2.76511238e+00 -6.92943997e+00]
 [-3.68543349e-01  1.53457576e+00 -3.03522931e+00]]


TODO -- Investigate whether the self-attention mechanism is covariant with respect to permutation.
If it is, when we permute the columns of the input matrix $\mathbf{X}$, the columns of the output matrix $\mathbf{X}'$ will also be permuted.

A: yes it is

In [44]:
X_permuted = X[:, [1, 0, 2]]

X_prime = scaled_dot_product_self_attention(X_permuted,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

Attention matrix:
[[1.86743853e-01 9.69367466e-01 8.77798694e-07]
 [3.29483974e-02 7.51694838e-15 9.99989399e-01]
 [7.80307749e-01 3.06325336e-02 9.72336127e-06]]

[[ 1.37185441e+00  9.67678489e-01 -2.71092104e+00]
 [ 6.21096251e-03 -2.38858164e-01  3.55377970e+00]
 [ 2.76511238e+00 -7.69157406e-01 -6.92943997e+00]
 [ 1.53457576e+00 -3.68543349e-01 -3.03522931e+00]]
